In [0]:
#importando os modulos

import numpy
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, BatchNormalization, Activation
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.constraints import maxnorm
from keras.utils import np_utils

In [0]:
seed = 21

In [0]:
#O CIFAR-10 é um conjunto de dados de imagens grandes contendo mais de 60.000 imagens representando 10 classes diferentes de objetos, como gatos, aviões e carros.

from keras.datasets import cifar10 # importando os dados CIFAR10 

In [0]:
#Carregando os conjutos de dados do dataset CIFAR10

(X_train, y_train), (X_test, y_test) = cifar10.load_data()

In [0]:
#Transformando os dados em um tipo flutuando. As imagens estão com valores de 255. Devemos dividir por esse valor para normalizalos 
#Assim não estaremos sobracarregando a rede de aprendizado 

# normalize the inputs from 0-255 to between 0 and 1 by dividing by 255
    
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train = X_train / 255.0
X_test = X_test / 255.0

In [0]:
#As imagens precisam ser codificadas primeiro
#Usaremos a codificação binaria para esse caso
# one hot encode outputs

y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
class_num = y_test.shape[1]

In [0]:
y_train[2]

In [0]:
#Criando o modelo de aprendizado do Keras
model = Sequential()

Ao implementar estes em Keras, temos que especificar o número de canais 
O filtros que queremos (que é o 32 abaixo pois as imagens são deste tamanho)
o tamanho do filtro que queremos (3 x 3, neste caso), a forma de entrada (ao criar a primeira camada ) e a ativação e preenchimento que precisamos.

In [0]:
#A primeira camada do nosso modelo é uma camada convolucional. 
#Ele irá receber as entradas e executar filtros convolucionais nelas.

model.add(Conv2D(32, (3, 3), input_shape=X_train.shape[1:], padding='same')) #padding='same'significa apenas que não estamos alterando o tamanho da imagem:
model.add(Activation('relu'))

Agora faremos uma camada de dropout para evitar overfitting, que funciona eliminando aleatoriamente algumas das conexões entre as camadas (0.2 significa que cai 20% das conexões existentes):

In [0]:
model.add(Dropout(0.2))

Também podemos querer fazer a normalização em lote aqui. A normalização em lote normaliza as entradas indo para a próxima camada, garantindo que a rede sempre crie ativações com a mesma distribuição que desejamos

In [0]:
model.add(BatchNormalization())

Agora vem outra camada convolucional, mas o tamanho do filtro aumenta para que a rede possa aprender representações mais complexas:

In [0]:
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))

Aqui está a camada de pooling, como discutido antes, isso ajuda a tornar o classificador de imagem mais robusto, para que ele possa aprender padrões relevantes. Há também a eliminação e a normalização do lote:

In [0]:
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(BatchNormalization())

Esse é o fluxo básico para a primeira metade de uma implementação da CNN: Convolutional, ativação, dropout, pooling .

É importante não ter muitas camadas de pooling, pois cada pool descarta alguns dados. 

In [0]:
#Quanto mais camadas na rede mais representações para trabalhar

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(BatchNormalization())

In [0]:
model.add(Conv2D(128, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())

In [0]:
model.add(Flatten())
model.add(Dropout(0.2))

Agora fazemos uso da Denseimportação e criamos a primeira camada densamente conectada. Precisamos especificar o número de neurônios na camada densa. Observe que o número de neurônios nas camadas seguintes diminui, aproximando-se do mesmo número de neurônios que existem no conjunto de dados (neste caso, 10). A restrição do kernel pode regularizar os dados à medida que aprende, outra coisa que ajuda a evitar o overfitting. É por isso que nós importamos maxnormmais cedo.

In [0]:
model.add(Dense(256, kernel_constraint=maxnorm(3)))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())

In [0]:
model.add(Dense(128, kernel_constraint=maxnorm(3)))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())

Finalmente, a softmaxfunção de ativação seleciona o neurônio com a maior probabilidade como saída, votando que a imagem pertence a essa classe:

In [0]:
model.add(Dense(class_num))
model.add(Activation('softmax'))

O otimizador é o que ajustará os pesos em sua rede para se aproximar do ponto de menor perda. O Adamalgoritmo é um dos otimizadores mais comumente usados, pois oferece ótimo desempenho na maioria dos problemas.

Necessitamos escolher um numero de épocas (tempo) de treino para nossa rede. Se escolhermos um numero muito baixo de épocas, nosso algoritmo não convirjará para um mínimo global. Se escolhermos um numero muito grande, correremos o risco de nosso algoritmo se acostumar demais aos dados de teste, e terá má performance em dados reais

In [0]:
epochs = 25
optimizer = 'adam'

Vamos agora compilar o modelo com nossos parâmetros escolhidos. Vamos também especificar uma métrica para usar.

In [0]:
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [26]:
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 64)        18496     
_________________________________________________________________
activation_2 (Activation)    (None, 32, 32, 64)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 64)        0         
__________

Agora nós começamos a treinar o modelo. Para fazer isso, tudo o que precisamos fazer é chamar a fit()função no modelo e passar os parâmetros escolhidos.

In [27]:
numpy.random.seed(seed)
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, batch_size=64)

W0731 21:31:09.335075 139872616568704 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 50000 samples, validate on 10000 samples
Epoch 1/25
50000/50000 [==============================] - 19s 388us/step - loss: 1.5100 - acc: 0.4606 - val_loss: 1.2841 - val_acc: 0.5304
Epoch 2/25
50000/50000 [==============================] - 14s 285us/step - loss: 1.0490 - acc: 0.6308 - val_loss: 0.8584 - val_acc: 0.6918
Epoch 3/25
50000/50000 [==============================] - 14s 284us/step - loss: 0.8576 - acc: 0.7003 - val_loss: 0.7947 - val_acc: 0.7208
Epoch 4/25
50000/50000 [==============================] - 14s 285us/step - loss: 0.7540 - acc: 0.7358 - val_loss: 0.6775 - val_acc: 0.7639
Epoch 5/25
50000/50000 [==============================] - 14s 285us/step - loss: 0.7002 - acc: 0.7540 - val_loss: 0.6500 - val_acc: 0.7696
Epoch 6/25
50000/50000 [==============================] - 14s 285us/step - loss: 0.6566 - acc: 0.7704 - val_loss: 0.6230 - val_acc: 0.7840
Epoch 7/25
50000/50000 [==============================] - 14s 287us/step - loss: 0.6233 - acc: 0.7815 - val_loss: 0.

Agora podemos avaliar o modelo e ver como ele é executado. Basta usar model.evaluate():

In [28]:
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 83.14%
